In [1]:
import importlib
import sys
import requests
import time
import json
import pandas as pd

sys.path.append('..')
import ash

importlib.reload(ash)

<module 'ash' from '/Users/cjan/golark/ash/tests/../ash/__init__.py'>

In [2]:
from ash.server import ASHModel
m = ASHModel('../models/qwen2.5-coder-3b-instruct-q4_k_m.gguf')
m.load()

🤖 Loading local model: ../models/qwen2.5-coder-3b-instruct-q4_k_m.gguf


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

✅ Local model loaded successfully in 2.28 seconds!
🔥 Warming up model...
✅ Model warmed up in 14.55 seconds!


In [3]:
m.generate_command("remove file a")

'rm a'

In [ ]:
total_points = 0
earned_points = 0
results = []

print("Starting AI command generation tests...")
print("=" * 50)

# Load test cases from JSON file
with open('./test_data/test_cases_basic.json', 'r') as f:
    test_data = json.load(f)
    test_cases = test_data['tests']

# Initialize results list
results = []

# Test each case
for i, test_case in enumerate(test_cases, 1):
    query = test_case['query']
    expected = test_case['expected']
    points = test_case['points']
    
    total_points += points
    
    try:
        # Start timer for this test case
        start_time = time.time()
        
        # Call AI model
        ai_response = m.generate_command(query)
        
        # End timer and calculate duration
        end_time = time.time()
        duration = end_time - start_time
        
        # Simple comparison - check if expected command is in AI response
        # This handles cases where AI might add arguments or variations
        if isinstance(ai_response, str):
            ai_response = ai_response.strip('"')
            ai_response = ai_response.strip('\'')

        # For each expected command, check if all its tokens (split by space) are present in the ai_response, regardless of order.
        def command_tokens_match(expected_cmd, ai_cmd):
            # Remove extra quotes and normalize whitespace
            expected_tokens = expected_cmd.replace('"', '').replace("'", '').split()
            ai_tokens = ai_cmd.replace('"', '').replace("'", '').split()
            # All expected tokens must be present in ai_response tokens (order doesn't matter)
            return all(token in ai_tokens for token in expected_tokens)

        is_correct = any(command_tokens_match(str(e), ai_response) for e in expected)

        results.append({
            'query': query,
            'expected': expected,
            'ai_response': ai_response,
            'pass/fail': 'PASS' if is_correct else 'FAIL',
            'points_earned': points if is_correct else 0,
            'duration_seconds': round(duration, 3)
        })

        #print(f"Test {i}: {query} | {expected} | {ai_response} - {duration:.3f}s - {'PASS' if is_correct else 'FAIL'}")
        
    except Exception as e:
        print(f"✗ ERROR: {e}")
        
    # Small delay to avoid overwhelming the API
    time.sleep(0.05)

# Convert results to pandas dataframe
results_df = pd.DataFrame(results)

# Sum points earned
points_earned = results_df['points_earned'].sum()
score_fraction = points_earned / total_points if total_points > 0 else 0
total_duration = results_df['duration_seconds'].sum()


Starting AI command generation tests...
Test 1: home | ['cd ~', 'cd $HOME'] | cd ~ - 10.674s - PASS
Test 2: go home | ['cd ~', 'cd $HOME'] | cd ~ - 9.701s - PASS
Test 3: change directory to home | ['cd ~', 'cd $HOME'] | cd ~ - 1.418s - PASS
Test 4: change to home directory | ['cd ~', 'cd $HOME'] | cd ~ - 1.229s - PASS
Test 5: create a new file called a | ['touch a', '> a'] | touch a - 2.864s - PASS
Test 6: move file a.txt to b.txt | ['mv a.txt b.txt'] | mv a.txt b.txt - 19.439s - PASS
Test 7: move a.txt to upper directory | ['mv a.txt ..'] | mv a.txt .. - 16.379s - PASS
Test 8: copy a to upper directory | ['cp a ..'] | mv a .. - 25.478s - FAIL
Test 9: list files | ['ls', 'ls -l', 'ls -la'] | ls - 8.343s - PASS
Test 10: show current directory | ['pwd'] | pwd - 7.466s - PASS
Test 11: create directory called x | ['mkdir x'] | mkdir x - 10.796s - PASS
Test 12: create directory called x if it doesnt exist | ['mkdir -p x'] | mkdir -p x - 17.599s - PASS
Test 13: remove a | ['rm -rf a', 'rm a'

In [6]:
results_df['ai_response'] = results_df['ai_response'].str[:40]

failed_df = results_df[results_df['pass/fail'] == 'FAIL']
if not failed_df.empty:
    print("\nFailed Tests:")
    print(failed_df[['query', 'expected', 'ai_response', 'pass/fail']].to_string(index=False))
else:
    print("\nAll tests passed!")




Failed Tests:
                       query                           expected         ai_response pass/fail
   copy a to upper directory                          [cp a ..]             mv a ..      FAIL
 search for aba txt in files [grep aba *.txt, grep 'aba' *.txt] grep -r "aba txt" .      FAIL
search for aba text in files [grep aba *.txt, grep 'aba' *.txt]     grep -r "aba" .      FAIL


In [14]:
avg_inference_time = results_df['duration_seconds'].mean()
print(f"\nAverage inference time: {avg_inference_time:.3f} seconds")

print(f"\nScore: {points_earned}/{total_points} ({score_fraction:.2%})")

display(results_df)



Average inference time: 5.177 seconds

Score: 105/114 (92.11%)


,query,expected,ai_response,pass/fail,points_earned,duration_seconds
0,home,"[cd ~, cd $HOME]",cd ~,PASS,3,10.674
1,go home,"[cd ~, cd $HOME]",cd ~,PASS,3,9.701
2,change directory to home,"[cd ~, cd $HOME]",cd ~,PASS,3,1.418
3,change to home directory,"[cd ~, cd $HOME]",cd ~,PASS,3,1.229
4,create a new file called a,"[touch a, > a]",touch a,PASS,3,2.864
5,move file a.txt to b.txt,[mv a.txt b.txt],mv a.txt b.txt,PASS,3,19.439
6,move a.txt to upper directory,[mv a.txt ..],mv a.txt ..,PASS,3,16.379
7,copy a to upper directory,[cp a ..],mv a ..,FAIL,0,25.478
8,list files,"[ls, ls -l, ls -la]",ls,PASS,3,8.343
9,show current directory,[pwd],pwd,PASS,3,7.466
